In [1]:
# !pip install Praw
# !pip install --upgrade praw

In [2]:
import sys
# !{sys.executable} -m pip install praw
# !{sys.executable} -m pip install psaw
import praw
import pandas as pd
import datetime as dt
import json
import psaw
from psaw import PushshiftAPI

f = open("secret_python.txt", "r")
client_id = f.readline().rstrip()
secret_key = f.readline().rstrip()
f.close()

In [3]:
reddit = praw.Reddit(client_id=client_id, \
                     client_secret=secret_key, \
                     user_agent='Scrape')

reddit.read_only

True

In [4]:
#Enter time_start, time_end as strings in format MM/DD/YYYY
def set_epochs(time_start, time_end):
    start_date = dt.datetime.strptime(time_start, "%m/%d/%Y")
    end_date = dt.datetime.strptime(time_end, "%m/%d/%Y")
    
    start_epoch = int(dt.datetime.timestamp(start_date))
    end_epoch = int(dt.datetime.timestamp(end_date))
    return((start_epoch, end_epoch))

In [5]:
def get_search_results_by_sub(subr, start_epoch, end_epoch):
    api = PushshiftAPI(reddit)
    searchResults = api.search_submissions(after=start_epoch,
                            before=end_epoch,
                            subreddit=subr)
    return list(searchResults)

def make_dict_from_search(cache):
    maxUpvote = 0
    iterator = 1
    memesDict = {}
    memesDict["_default"] = {}
    for submission in cache:
        if (submission is not None and submission.is_video is False and submission.removed_by is None):
            if (submission.ups > maxUpvote):
                maxUpvote = submission.ups
            memesDict["_default"][iterator] = {"title":submission.title, "thumbnail":{"thumbnail":submission.thumbnail, "height":submission.thumbnail_height, "width":submission.thumbnail_width}, "created_utc":submission.created_utc, "id":submission.id, "ups":submission.ups, "downs":submission.downs, "media": submission.url, "url": submission.permalink, "awards": submission.all_awardings, "subreddit":submission.subreddit_name_prefixed, "subscribers": submission.subreddit_subscribers, "is_nsfw": submission.over_18}
            if (submission.author is not None):
                memesDict["_default"][iterator]["author"] = submission.author.name 
            else:
                memesDict["_default"][iterator]["author"] = "N/A"
            iterator +=1
    return memesDict

In [6]:
#Feel free to add to list of memes
subreddits = ["MemeEconomy", "memes", "me_irl", "dank_meme", "dankmeme"]

In [7]:
def batch_scrape(time_start, time_end, file_name, subreddits):
    start_epoch, end_epoch = set_epochs(time_start, time_end)
    
    meme_dicts = []
    for sub in subreddits:
        meme_dicts.append(make_dict_from_search(get_search_results_by_sub(sub,start_epoch, end_epoch)))
        
    with open(file_name+'.json', 'w') as json_file:
        json.dump(meme_dicts, json_file)

In [10]:
batch_scrape("3/20/2020", "3/23/2020", "data/batch_two", subreddits)